In [36]:
def cleanup_data_values_return_float(data_in, on_error_return_runtimerror=True):
    '''
    Description:
        Checks raw data quality, convert if required
        Remove and converts commas and dots, to assure decimal point usage but removal of thousand seperators
        Raises errors if datatype not useable

    Parameters:
        Raw data in
        Boolean to determine whether program should give runtimeerrors upon encountering error (default True)

    Returns:
        Floating point number of input
        Errormessage in case wrong datatype
    '''
    message_error_string = " is the datatype value in database, but it must be a floating point or integer"

    if type(data_in) is str:
        data_in = data_in.replace(',','.')
        if data_in.count('.') > 1:
            data_in = data_in.replace('.','', data_in.count('.') - 1)

    try:
        data_in = float(data_in)
    except:
        if on_error_return_runtimerror:
            if type(data_in) is bool:
                raise RuntimeError("Boolean " + message_error_string)
            if type(data_in) is str:
                raise RuntimeError("String " + message_error_string)

    return data_in


def cleanup_dictionary_values_convert_numbers_to_float_and_indicated_strings_to_function(dict, string_indicator):
    '''
    Description:
        Cleans up the dictionary values
        Remove and converts commas and dots, to assure decimal point usage but removal of thousand seperators
        Floating point number if number type
        In case there is a string (indicated by startswith), the string is converted to a function if possible
        Does not raise errors if datatype not useable but just continues

    Parameters:
        Dictionary it relates to
        Indicative starting part of a string to indicate it is a function

    Returns:
        Cleaned up dictionary 

    Depending on other functions:
        Depends on function cleanup_data_values_return_float
    '''
    for item in list(dict.keys()):
        try:
            # cleanup the style of numbering and decimal points
            dict[item] = cleanup_data_values_return_float(dict[item])
        except:
            try:
                if type(dict[item]) is str and dict[item].startswith(string_indicator):
                    dict[item] = eval(dict[item]) # updates value to a 'pointer' of the actual formula
            except:
                pass
            pass
    return


def remove_dots_from_string(input_string, on_wrong_datatype_return_errormessage_string=False):
    '''
    Description:
        Removes dots from an input string
        Checks whether data type is string otherwise returns input
    
    Parameters:
        Input string
        In case the datatype is not a string, return error or not (default False)

    Returns:
        Input String, cleaned up if possible
    '''
    message_error_string = " is the datatype value supplied, but it must be a string datatype"
    
    if type(input_string) == str:
        return input_string.replace('.','')
    else:
        if on_wrong_datatype_return_errormessage_string:
            raise RuntimeError(str(type(input_string)) + message_error_string)

    return input_string


def convert_units_from_dict(dict_to_use, unit_subject, unit_system, unit_specs, data_in, on_error_return_runtimerror=True, on_wrong_datatype_return_string=True):
    '''
    Description:
        Retrieves conversion units from dictionary to be multiplied
        Apply a function in case it is a function
        Multiplies a factor if it is a factor
        
    Parameters:
        dict_to_use = dictionary used to retrieve value
        unit_subject = describes the subject of unit, e.g. temperature, length
        unit_system = describes the unit systeml USCS, Imperial, SI etc
        unit_specs = Specificies the exact unit used
        In case conversion format does not exist in dictionary, this means the conversion list might be wrong or not fulfilling the dataset thus it returns a runtimeerror (default True)
        In case wrong datatype (so not a number to convert) return error strin, handy for storing an error in the dataset (default True)

    Returns:
        Converted value of data_in
    '''
    retrieval_value = unit_subject + '_' + unit_system + '_' + unit_specs
    if on_error_return_runtimerror:            
        if retrieval_value not in dict_to_use:
            # This will stop the program!
            raise RuntimeError("Unit "  + retrieval_value + " not found in dictionary. Please update data or dictionary.")

    x = dict_to_use[retrieval_value]
    try:
        if callable(x):
            return x(data_in) # applies the function as stated in dictionary
        else:
            return data_in * x # conversion is not a number, e.g. a function
    except:
        if on_wrong_datatype_return_string:
            return str(type(data_in)) + ' datatype was supplied and seems to be incorrect, it should be a float or integer.'
        else:
            pass
    return

def import_new_supplier(csv_filename = '.\Data.csv', csv_has_header = None, csv_seperator = ';', csv_encoding = 'utf-8', csv_low_memory = False, 
columns_not_useable = [range(1,7), range(8,9), range(10,14)], column_with_product_id = 0, system_name_for_product_id = '12NC', 
row_num_unit_subject = 0, row_num_unit_system = 1, row_num_unit_specs = 2, row_num_data_starts = 6):
    '''
    Description:
        Facilitates the import of a new supplier throug a CSV file and cleans the dataset:
            1. Transforms CSV to Pandas DataFrame
            2. Removes unneccessary columns
            3. Converts numbers to correct unit, cleans up string values
            4. Removes metadata rows from dataframe

    Parameters:
        Standard python pandas dataframe input parameters:
            csv_filename = pathname as string (default = '.\Data.csv')
                describes the filepath of the to be imported dataset
            csv_has_header = int, list of int or None (default = None)
                indicates whether dataset contains a header for column name information
            csv_seperator = string (default = ';')
                the seperator used in the csv file for column identification purposes
            csv_encoding = string (default = 'utf-8')
                Encoding system used in the csv
            csv_low_memory = boolean (default = False)
                Deterement to let go memory restrictions for large datasets

        Dataset specifics:
            columns_not_useable = list (default = [range(1,7), range(8,9), range(10,14)])

            column_with_product_id = (default = 0)

            system_name_for_product_id = (default = '12NC')
            
            row_num_unit_subject = (default = 0)
            
            row_num_unit_system = (default = 1)
            
            row_num_unit_specs = (default = 2)
            
            row_num_data_starts = (default = 6)

    Returns:
        Cleaned up pandas dataframe

    Depends on:
        Functions:
        cleanup_data_values_return_float
        convert_units_from_dict
        remove_dots_from_string

    Adjustments inside code:
        There are a few booleans value where the other functiosn get called that might help during error handling:
            cleanup_data_values_return_float(x, on_error_return_runtimerror=False)
            convert_units_from_dict(dict_unit_csv, unit_subject, unit_system, unit_specs, x, on_error_return_runtimerror=True, on_wrong_datatype_return_string=True)
            remove_dots_from_string(x, on_wrong_datatype_return_errormessage_string=False)
        Please refer to documentation of individual functions for further documentation
    '''
    import pandas as pd
    df_import_supplier = pd.read_csv(csv_filename, header=csv_has_header, sep=csv_seperator, encoding=csv_encoding, low_memory=csv_low_memory)

    # drop unneccessary columns
    for col in columns_not_useable:
        try:
            df_import_supplier.drop(columns=col, axis=1, inplace=True)
        except:
            pass

    # clean up the database and by going over each column in the dataframe as each column might require different work
    for col in df_import_supplier.columns.tolist():
        # use metadata information from dataset to buildup the unit conversion information
        # this information is also used for column names later in code
        try:
            unit_subject = df_import_supplier[col].iloc[row_num_unit_subject].lower()
            unit_system = df_import_supplier[col].iloc[row_num_unit_system].upper()
            unit_specs = df_import_supplier[col].iloc[row_num_unit_specs]
        except:
            if col == column_with_product_id:
                unit_subject = system_name_for_product_id
            else:
                unit_subject = 'Properties not given by database please adjust'

        # Clean up all the individual data values to prepare them for storing to main database
        if col != column_with_product_id:
            df_import_supplier[col] = df_import_supplier[col].map(lambda x : cleanup_data_values_return_float(x, on_error_return_runtimerror=False))
            df_import_supplier[col] = df_import_supplier[col].map(lambda x : convert_units_from_dict(dict_unit_csv, unit_subject, unit_system, unit_specs, x, on_error_return_runtimerror=True, on_wrong_datatype_return_string=True))
        else:
            df_import_supplier[col] = df_import_supplier[col].map(lambda x: remove_dots_from_string(x, on_wrong_datatype_return_errormessage_string=False))

        # Rename columns
        df_import_supplier.rename(columns={col: unit_subject}, inplace=True)

    # Drop the rows which contain the metadata info
    df_import_supplier.drop(df_import_supplier.index[range(0, row_num_data_starts)], inplace=True) 

    return df_import_supplier

In [37]:
# Read csv to dictionary and cleanup dictionary values
import csv
reader = csv.reader(open('conv.csv', 'r'), delimiter=';')
dict_unit_csv = {} # start with empty dictionary

for k, v in reader:
    dict_unit_csv[k] = v # add key and value to dict

cleanup_dictionary_values_convert_numbers_to_float_and_indicated_strings_to_function(dict_unit_csv, 'lambda')

print(dict_unit_csv)

{'distance_SI_km': 0.001, 'distance_SI_m': 1.0, 'distance_SI_cm': 100.0, 'distance_SI_mm': 1000.0, 'distance_USCS_mi.': 1.609, 'distance_USCS_ft': 0.3048, 'distance_USCS_in': 0.0254, 'volume_USCS_cu_in': 0.016387064, 'volume_USCS_cu_ft': 28.0, 'volume_USCS_cu_yd': 765.0, 'volume_USCS_bbl': 159.0, 'volume_SI_L': 1.0, 'temperatures_USCS_°F': <function <lambda> at 0x000001D0A21C7048>, 'temperatures_USCS_F': <function <lambda> at 0x000001D0A21C73A8>, 'temperatures_SI_°K': <function <lambda> at 0x000001D0A21C74C8>, 'temperatures_SI_K': <function <lambda> at 0x000001D0A0179048>, 'temperatures_SI_°C': 1.0, 'temperatures_SI_C': 1.0, 'mass_USCS_lb': 0.45359237, 'mass_SI_kg': 1.0, 'mass_SI_g': 1000.0}


In [38]:
# Self-made Dictionary
# All conversions are made to SI units using https://en.wikipedia.org/wiki/United_States_customary_units#Units_of_length

dict_unit_hardcoded = {
    # Distance to Meter (SI) using * multiplication to go to SI
    'distance_SI_km':0.001, # kilometer
    'distance_SI_m':1, # meter
    'distance_SI_cm':100, # centimeter
    'distance_SI_mm':1000, # millimeter
    'distance_USCS_mi.':1609.344, # miles
    'distance_USCS_ft':0.3048, # feet
    'distance_USCS_in':0.0254, # inch
    # Volume to Liter (SI) using * multiplication to go to SI
    'volume_USCS_cu_in':0.016387064, # cubic inch
    'volume_USCS_cu_ft':28.316846592, # cubic feet
    'volume_USCS_cu_yd':764.554857984, # cubic yard
    'volume_USCS_bbl':158.987294928, # oil barrel
    'volume_SI_L':1, # Liter
    # Temperatures
    'temperatures_USCS_°F': lambda x : ((5/9) * (x - 32)), # Fahrenheit to C
    'temperatures_USCS_F': lambda x : ((5/9) * (x - 32)), # Fahrenheit to C
    'temperatures_SI_°K': lambda x : (x - 273.15), # Kelvin
    'temperatures_SI_K': lambda x : (x - 273.15), # Kelvin
    'temperatures_SI_°C': 1, # Celsius
    'temperatures_SI_C': 1, # Celsius
    # Weights
    'mass_USCS_lb': 0.45359237, # Pounds
    'mass_SI_kg': 1, # Kilogram
    'mass_SI_g': 1000, # grams
}

In [74]:
import pandas as pd
df = pd.DataFrame()

df['id'] = [1, 2, 3]
df['temp'] = [10, 20, 30]
df['cost'] = [15, 30, 45]

df2 = pd.DataFrame()
df2['id'] = [1, 2, 4]
df2['temp'] = [10, 20, 30]
df2['price'] = [15, 30, 45]

#pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None,
#         left_index=False, right_index=False, sort=True,
#         suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)

df_ttl = pd.DataFrame()
df_tll = pd.merge(df, df2, on='id', left_index=False, right_index=False)

df.head()
df2.head()
df_ttl.head()

""


In [78]:
def verify_columns_of_dataframes(main_dataset, dataset_to_add, on_error_return_runtimerror=True):
    '''
    Description:
        Checks raw data quality, convert if required
        Remove and converts commas and dots, to assure decimal point usage but removal of thousand seperators
        Raises errors if datatype not useable

    Parameters:
        Raw data in
        Boolean to determine whether program should give runtimeerrors upon encountering error (default True)

    Returns:
        Floating point number of input
        Errormessage in case wrong datatype
    '''
    
    message_error_string = " is the datatype value in database, but it must be either a list or a pandas dataframe"

    if type(main_dataset) == pandas.core.frame.DataFrame:
        main_dataset_columns = main_dataset.columns.tolist()
    else:
        if type(main_dataset) != list:
            if on_error_return_runtimerror:
                if type(main_dataset) is bool:
                    raise RuntimeError("Boolean " + message_error_string)
                if type(main_dataset) is str:
                    raise RuntimeError("String " + message_error_string)

    for col in main_dataset_columns:
        if col in cols_of_import:
            return 'this col is in both : ' + str(col)
        else:
            return 'this col is not in both : ' + str(col)

    return str(type(main_dataset) + type(datset_to_add))

import pandas as pd
df_main = pd.DataFrame(columns=['12NC','temperature','mass'])

df_import = import_new_supplier()
cols_of_import = df_import.columns.tolist()
type(cols_of_import)
type(df_import)
#verify_columns_of_dataframes(df_main, cols_of_import)

    #pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None,
    #         left_index=False, right_index=False, sort=True,
    #         suffixes=('_x', '_y'), copy=True, indicator=False,
    #         validate=None)

#df_main = df_main.append(['1', 1, 1])
#result = pd.concat([df_main, import_new_supplier()], axis=0, sort=False)
#df_main.head(10)



pandas.core.frame.DataFrame

In [5]:
# General Parameters
csv_filename = 'Data.csv'
csv_has_header = None
csv_seperator = ';'
csv_encoding = 'utf-8'
csv_low_memory = False

columns_not_useable = [range(1,7), range(8,9), range(10,14)]
column_with_product_id = 0
system_name_for_product_id = '12NC'
row_num_unit_subject = 0
row_num_unit_system = 1
row_num_unit_specs = 2
row_num_data_starts = 6


# Importing a specific supplier
import pandas as pd

# import
df_import_supplier = pd.read_csv(csv_filename, header=csv_has_header, sep=csv_seperator, encoding=csv_encoding, low_memory=csv_low_memory)

# clean up the database and drop unneccessary columns
for col in columns_not_useable:
    try:
        df_import_supplier.drop(columns=col, axis=1, inplace=True)
    except:
        pass

# use information from dataset to buildup the unit conversion information
for col in df_import_supplier.columns.tolist():
    # Derive column names from database content, population of columns happens later
    try:
        unit_subject = df_import_supplier[col].iloc[row_num_unit_subject].lower()
        unit_system = df_import_supplier[col].iloc[row_num_unit_system].upper()
        unit_specs = df_import_supplier[col].iloc[row_num_unit_specs]
    except:
        if col == column_with_product_id:
            unit_subject = system_name_for_product_id
        else:
            unit_subject = 'Properties not given by database please adjust'

    # Clean up all the individual data values to prepare them for storing to main database
    if col != column_with_product_id:
        df_import_supplier[col] = df_import_supplier[col].map(lambda x : cleanup_data_values_return_float(x, on_error_return_runtimerror=False))
        df_import_supplier[col] = df_import_supplier[col].map(lambda x : convert_units_from_dict(dict_unit_csv, unit_subject, unit_system, unit_specs, x, on_error_return_runtimerror=True, on_wrong_datatype_return_string=True))
    else:
        df_import_supplier[col] = df_import_supplier[col].map(lambda x: remove_dots_from_string(x, on_wrong_datatype_return_errormessage_string=False))

    # Rename columns
    df_import_supplier.rename(columns={col: unit_subject}, inplace=True)

# Drop all the information that is from here not neccessary
df_import_supplier.drop(df_import_supplier.index[range(0, row_num_data_starts)], inplace=True) 


df_import_supplier.head(10)

,12NC,temperatures,mass
6,402255334708,15,89.8113
7,402258698328,1.11111,183.705
8,402249767239,25,698.986
9,402235950496,55,311.618
10,402264653803,85.5556,306.175
11,402210410701,13.8889,740.263
12,402262421235,82.7778,406.872
13,402224951198,16.6667,759.767
14,402254974878,38.8889,801.044
15,402257850893,42.2222,137.438


In [11]:
# General importing
import pandas as pd

# import
df_import_supplier_A = pd.DataFrame()
df_import_supplier_A = pd.read_csv("Data.csv", header=None, sep=';', encoding='utf-8', low_memory=False)

# Clean up the database
# drop column
try:
    df_import_supplier_A = df_import_supplier_A.drop(columns=range(1,7), axis=1)
    df_import_supplier_A = df_import_supplier_A.drop(columns=range(8,9), axis=1)
    df_import_supplier_A = df_import_supplier_A.drop(columns=range(10,14), axis=1)
except:
    pass

# Make the conversion list
df_products = pd.DataFrame() # This can become a loading option later
'''
df_products["12NC"] = df_import_supplier_A[0]
df_products = df_products.dropna()
df_products = df_products.set_index("12NC")
'''
df_import_supplier_A.head(8)

,0,7,9
0,NaN,Temperatures,Mass
1,NaN,USCS,USCS
2,NaN,°F,lb
3,NaN,32,0
4,NaN,200,2000
5,12NC,°F,Lb
6,402.255.334.708,59,198
7,402.258.698.328,34,405


In [12]:
for col in df_import_supplier_A.columns.tolist():
    # Define the types as used in columns derived from database
    try:
        unit_subject = df_import_supplier_A[col].iloc[0].lower()
        unit_system = df_import_supplier_A[col].iloc[1].upper()
        unit_specs = df_import_supplier_A[col].iloc[2]
    except:
        unit_subject = "12NC"

    # Add columns and convert in place
    new_col_name = str('Supplier A '+ unit_subject)
    df_products[new_col_name] = df_import_supplier_A[col].iloc[6:df_import_supplier_A.shape[0]]

    if col > 0:
        df_products[new_col_name] = df_products[new_col_name].apply(lambda x : cleanup_data_values_return_float(x))
        df_products[new_col_name] = df_products[new_col_name].apply(lambda x : convert_units_from_dict(dict_unit_csv, unit_subject, unit_system, unit_specs, x))

    elif col == 0:
        df_products[new_col_name] = df_products[new_col_name].apply(lambda x : x.replace('.',''))

df_products = df_products.set_index("Supplier A 12NC")
df_products.head()

,Supplier A temperatures,Supplier A mass
Supplier A 12NC,,
402255334708,15.000000,89.811289
402258698328,1.111111,183.704910
402249767239,25.000000,698.985842
402235950496,55.000000,311.617958
402264653803,85.555556,306.174850


In [ ]:
# START SCREWING AROUND HERE

In [13]:
# General importing
import pandas as pd

# make DB with converted values
df_main = pd.DataFrame()
df_main

""


In [27]:
# Importing a specific supplier
import pandas as pd

# import
df_import_supplier_A = pd.DataFrame()
df_import_supplier_A = pd.read_csv("Data.csv", header=None, sep=';', encoding='utf-8', low_memory=False)

# Clean up the database
# drop column
try:
    df_import_supplier_A = df_import_supplier_A.drop(columns=range(1,7), axis=1)
    df_import_supplier_A = df_import_supplier_A.drop(columns=range(8,9), axis=1)
    df_import_supplier_A = df_import_supplier_A.drop(columns=range(10,14), axis=1)
except:
    pass

df_temp = pd.DataFrame()

# Use information from dataset to buildup the unit conversion information
for col in df_import_supplier_A.columns.tolist():
    # Define the types as used in columns derived from database
    try:
        unit_subject = df_import_supplier_A[col].iloc[0].lower()
        unit_system = df_import_supplier_A[col].iloc[1].upper()
        unit_specs = df_import_supplier_A[col].iloc[2]
    except:
        unit_subject = "12NC" # if not possible it must be the 12NC

    # Add columns and convert in place
    # From row 6 on there is information
    data_starts_at_row_num = 6
    df_temp[unit_subject] = df_import_supplier_A[col].iloc[data_starts_at_row_num:df_import_supplier_A.shape[0]]

    if col > 0:
        df_temp[unit_subject] = df_temp[unit_subject].map(lambda x : cleanup_data_values_return_float(x))
        df_temp[unit_subject] = df_temp[unit_subject].map(lambda x : convert_units_from_dict(dict_unit_csv, unit_subject, unit_system, unit_specs, x))

    elif col == 0:
        df_temp[unit_subject] = df_temp[unit_subject].map(lambda x : x.replace('.',''))

#df_temp = df_temp.set_index("Supplier A 12NC")
df_temp.head(10)

,12NC,temperatures,mass
6,402255334708,15.000000,89.811289
7,402258698328,1.111111,183.704910
8,402249767239,25.000000,698.985842
9,402235950496,55.000000,311.617958
10,402264653803,85.555556,306.174850
11,402210410701,13.888889,740.262748
12,402262421235,82.777778,406.872356
13,402224951198,16.666667,759.767220
14,402254974878,38.888889,801.044125
15,402257850893,42.222222,137.438488


In [37]:
def add_suppliers_to_main_database(csv_file, seperator =';', )



# Importing a specific supplier
import pandas as pd

# import
df_import_supplier_A = pd.DataFrame()
df_import_supplier_A = pd.read_csv("Data.csv", header=None, sep=seperator, encoding='utf-8', low_memory=False)

# Clean up the database
# drop column
try:
    df_import_supplier_A = df_import_supplier_A.drop(columns=range(1,7), axis=1)
    df_import_supplier_A = df_import_supplier_A.drop(columns=range(8,9), axis=1)
    df_import_supplier_A = df_import_supplier_A.drop(columns=range(10,14), axis=1)
except:
    pass

df_temp = pd.DataFrame()

# Use information from dataset to buildup the unit conversion information
for col in df_import_supplier_A.columns.tolist():
    # Define the types as used in columns derived from database
    try:
        unit_subject = df_import_supplier_A[col].iloc[0].lower()
        unit_system = df_import_supplier_A[col].iloc[1].upper()
        unit_specs = df_import_supplier_A[col].iloc[2]
    except:
        unit_subject = "12NC" # if not possible it must be the 12NC

    # Add columns and convert in place
    # From row 6 on there is information
    data_starts_at_row_num = 6

    df_temp[unit_subject] = df_import_supplier_A[col].iloc[data_starts_at_row_num:df_import_supplier_A.shape[0]]

    if col > 0:
        df_temp[unit_subject] = df_temp[unit_subject].apply(lambda x : cleanup_data_values_return_float(x))
        df_temp[unit_subject] = df_temp[unit_subject].apply(lambda x : convert_units_from_dict(dict_unit_csv, unit_subject, unit_system, unit_specs, x))

    elif col == 0:
        df_temp[unit_subject] = df_temp[unit_subject].apply(lambda x : x.replace('.',''))

#df_temp = df_temp.set_index("Supplier A 12NC")
df_temp.head()

SyntaxError: invalid syntax (<ipython-input-37-6e0674773da1>, line 1)

In [ ]:
# STOPPED SCREWING AROUND HERE

In [7]:
# import
df_import_supplier_B = pd.DataFrame()
df_import_supplier_B = pd.read_csv("DataB.csv", header=None, sep=';', encoding='utf-8', low_memory=False,dtype=str)

# Clean up the database
# drop column
df_import_supplier_B = df_import_supplier_B.drop(columns=range(1,7), axis=1)
df_import_supplier_B = df_import_supplier_B.drop(columns=range(8,9), axis=1)
df_import_supplier_B = df_import_supplier_B.drop(columns=range(10,14), axis=1)

df_import_supplier_B.head()

,0,7,9
0,NaN,Temperatures,Mass
1,NaN,USCS,USCS
2,NaN,F,lb
3,NaN,80,0
4,NaN,24,2000


In [8]:
df_products.append(df_import_supplier_B)
print(df_products)

Supplier A temperatures  Supplier A mass
Supplier A 12NC                                          
402255334708                   15.000000        89.811289
402258698328                    1.111111       183.704910
402249767239                   25.000000       698.985842
402235950496                   55.000000       311.617958
402264653803                   85.555556       306.174850
...                                  ...              ...
402245489916                   68.888889       141.520819
402255946517                   36.111111       738.901971
402262790749                   47.777778       286.216785
402278043281                   48.333333       801.044125
402212761490                   35.555556       360.605934

[199994 rows x 2 columns]


In [9]:

# Make the conversion list
#df_products = pd.DataFrame() # This can become a loading option later
print(df_import_supplier_B.columns.tolist())
for col in df_import_supplier_B.columns.tolist():
    # Define the types as used in columns derived from database
    try:
        unit_subject = df_import_supplier_B[col].iloc[0].lower()
        unit_system = df_import_supplier_B[col].iloc[1].upper()
        unit_specs = df_import_supplier_B[col].iloc[2]
    except:
        unit_subject = "12NC"

    # Add columns and convert in place
    new_col_name = str('Supplier B '+ unit_subject)
    df_products[new_col_name] = df_import_supplier_B[col].iloc[6:df_import_supplier_B.shape[0]]

    if col > 0:
        df_products[new_col_name] = df_products[new_col_name].apply(lambda x : cleanup_data_values_return_float(x))
        df_products[new_col_name] = df_products[new_col_name].apply(lambda x : convert_units_from_dict(dict_unit_hardcoded, unit_subject, unit_system, unit_specs, x))
        
    elif col == 0:
        df_products[new_col_name] = df_products[new_col_name].apply(lambda x : str(x).replace('.',''))

df_products.head()

[0, 7, 9]


,Supplier A temperatures,Supplier A mass,Supplier B 12NC,Supplier B temperatures,Supplier B mass
Supplier A 12NC,,,,,
402255334708,15.000000,89.811289,nan,NaN,NaN
402258698328,1.111111,183.704910,nan,NaN,NaN
402249767239,25.000000,698.985842,nan,NaN,NaN
402235950496,55.000000,311.617958,nan,NaN,NaN
402264653803,85.555556,306.174850,nan,NaN,NaN


In [9]:
df_import_supplier_B.iloc[10]

0    402261157747
7     74,14683407
9            1575
Name: 10, dtype: object

In [44]:
df_import_supplier_B

,0,7,9
0,NaN,Temperatures,Mass
1,NaN,USCS,USCS
2,NaN,F,lb
3,NaN,80,0
4,NaN,24,2000
...,...,...,...
199995,402288016975,"89,91617234",1776
199996,402287657683,"100,9163358",1287
199997,402262726583,"82,06021872",729
199998,402220423670,"53,0804307",1850


In [ ]:
'''
Reserved space to use like notepad, handy to retrieve useful code
'''

## Here more code needs to come
'''
#######
#   PintPy
# convert_pintpy(1,"meter","cm") # does not work
ureg.meter
#   PyPi: Unit-converter
# #convert_unit_converter(100, kilometres, miles) # does not work
#   PyPi: Workdays
#   PyPi: BusinessHours
#######
'''

from datetime import datetime
start_time = datetime.now() # for time loop
# START TIME LOOP #
# END OF TIME LOOP #
time_elapsed = datetime.now() - start_time
print('Time elapsed (h:m:s.ms) {}'.format(time_elapsed))

In [41]:
## Trying out stuff

y = df_import_supplier_A['Temperatures']
y.iloc[2:y.size]

2          0
3         80
4          C
5         68
6         58
          ..
199994     5
199995    72
199996     5
199997    26
199998     5
Name: Temperatures, Length: 199997, dtype: object

In [ ]:
>>> from unit_converter.converter import convert, converts
>>>
>>> convert('2.78 daN*mm^2', 'mN*µm^2')
>>> Decimal('2.78E+10')
>>>
>>> converts('2.78 daN*mm^2', 'mN*µm^2')
>>> '2.78E+10'
>>>
>>> converts('78 min', 'h')
>>> '1.3'
>>>
>>> converts('52°C', '°F')
>>> '125.6'
>>>
>>> converts('120 km*h^-1', 'mile*h^-1')
>>> '74.56454306848007635409210214'

In [31]:
# General importing
import pandas as pd
from unit_converter.converter import convert, converts # copnvert returns decimal, converts returns string

dict_unit_for_pypi_unit_converter = {
    '°C':'°C',
    '°F':'°C',
    'lb':'kg'
}

# import
df_import_supplier_A = pd.DataFrame()
df_import_supplier_A = pd.read_csv("Data.csv", header=None, sep=';', encoding='utf-8', low_memory=False)

# Clean up the database
# drop column
df_import_supplier_A = df_import_supplier_A.drop(columns=range(1,7), axis=1)
df_import_supplier_A = df_import_supplier_A.drop(columns=range(8,9), axis=1)
df_import_supplier_A = df_import_supplier_A.drop(columns=range(10,14), axis=1)

df_products = pd.DataFrame()

for col in df_import_supplier_A.columns.tolist():
    # Define the types as used in columns derived from database
    try:
        unit_subject = df_import_supplier_A[col].iloc[0].lower()
        unit_system = df_import_supplier_A[col].iloc[1].upper()
        unit_specs = df_import_supplier_A[col].iloc[2]
    except:
        unit_subject = "12NC"
    
    # Add columns and convert in place
    new_col_name = str('Supplier A '+ unit_subject)
    df_products[new_col_name] = df_import_supplier_A[col].iloc[6:df_import_supplier_A.shape[0]]
    try: 
        if col > 0 :
            df_products[new_col_name] = df_products[new_col_name].apply(lambda x : convert(str(x + '' + unit_specs), dict_unit_for_pypi_unit_converter[unit_specs]))
    except:
        df_products[new_col_name] = None
        pass

df_products = df_products.set_index("Supplier A 12NC")
df_products.head()

,Supplier A temperatures,Supplier A mass
Supplier A 12NC,,
402.255.334.708,15.0000000000000000000000000,None
402.258.698.328,1.1111111111111111111111111,None
402.249.767.239,25.0000000000000000000000000,None
402.235.950.496,55.0000000000000000000000000,None
402.264.653.803,85.5555555555555555555555555,None


In [5]:
# Convert Units
########
# PyPi: Unit-Converter https://pypi.org/project/unit-convert/


dict_unit_for_pypi_unit_convert = {
    'fahrenheit':'celsius',
    'F':'celsius',
    'lb':'kg'
}

def convert_unit_converter(unit_in, x):
    from unit_convert import UnitConvert
    unit_out = dict_unit_for_pypi_unit_convert[unit_in]
    
    print(unit_in)
    print(unit_out)

    unit_in = 'fahrenheit'
    attrs = {unit_in: x}
    return getattr(UnitConvert(**attrs), unit_out)

'''
# Examplary code how to use pypi:
# Yards + kilometres to miles
x = UnitConvert(yards=136.23, kilometres=60).miles
print(x)
# Bytes to terabytes
x = UnitConvert(b=19849347813875).tb
print(x)
'''

# General importing
import pandas as pd

# import
df_import_supplier_A = pd.DataFrame()
df_import_supplier_A = pd.read_csv("Data.csv", header=None, sep=';', low_memory=False)

# Clean up the database
# drop column
df_import_supplier_A = df_import_supplier_A.drop(columns=range(1,7), axis=1)
df_import_supplier_A = df_import_supplier_A.drop(columns=range(8,9), axis=1)
df_import_supplier_A = df_import_supplier_A.drop(columns=range(10,14), axis=1)

for col in df_import_supplier_A.columns.tolist():
    # Define the types as used in columns derived from database
    try:
        unit_subject = df_import_supplier_A[col].iloc[0].lower()
        unit_system = df_import_supplier_A[col].iloc[1].upper()
        unit_specs = df_import_supplier_A[col].iloc[2]
    except:
        unit_subject = "12NC"
    
    # Add columns and convert in place
    new_col_name = str('Supplier A '+ unit_subject)
    df_products[new_col_name] = df_import_supplier_A[col].iloc[6:df_import_supplier_A.shape[0]]
    if col > 0 :
        df_products[new_col_name] = df_products[new_col_name].apply(lambda x : convert_unit_converter(unit_specs, x))

df_products = df_products.set_index("Supplier A 12NC")
df_products.head()

#x = UnitConvert(yards=136.23, kilometres=60).miles
#print(x)

def convert_unit_converter_old(data_in, unit_in, unit_out):
    from unit_convert import UnitConvert
    return(UnitConvert(unit_in=data_in).unit_out)

NameError: name 'df_products' is not defined

In [51]:
# Convert Units
########
# PintPy: https://pint.readthedocs.io/en/0.11/

## Error: currently this does not want to work

import pint
ureg = pint.UnitRegistry() 

'''
# Examplary code how to use pintpy: 
print(3 * ureg.meter + 4 * ureg.cm)
'''

#print(0 * ureg.meter + 4 * ureg.ft)
print(0 * ureg.meter + 2 * ureg.ft)

pint_dict = {
    'meter' : ureg.meter,
    'feet' : ureg.ft
}

def convert_pintpy(data_in, unit_in, unit_out):
    ureg = pint.UnitRegistry()

    unit_in = "ureg." + unit_in # this string conversion seems to create error
    unit_out = "ureg." + unit_out # but how to create this variable otherwise?
    out = (0 * unit_out + data_in * unit_in)

    return(out)

0.6095999999999999 meter
